In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

# Chrome WebDriver 생성
driver = webdriver.Chrome()

# 웹 페이지로 이동
driver.get("https://www.naver.com")

# Setting
loc = "구의동"

# 로드 될 때까지 대기
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

# 지도 클릭
map_icon = driver.find_elements(By.CLASS_NAME, "shortcut_item")[-4].click()

# 로드 될 때까지 대기
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

# 최근에 열린 Window 창으로 전환
# print(driver.window_handles)
driver.switch_to.window(driver.window_handles[-1])

# 키워드 입력 및 검색
input_box = driver.find_element(By.CLASS_NAME, "input_search")
input_box.send_keys(f"{loc} 맛집")
input_box.send_keys(Keys.ENTER)

time.sleep(2)

# iframe에 접근
iframe = driver.find_element(By.ID, "searchIframe")
driver.switch_to.frame(iframe)

# 스크롤 다운
def scroll_down(container):
    before_top = driver.execute_script("return arguments[0].scrollTop", container)

    while True:
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", container)
        after_top = driver.execute_script("return arguments[0].scrollTop", container)

        if int(before_top) == int(after_top):
            break
        
        before_top = after_top 

scroll_container = driver.find_element(By.ID, "_pcmap_list_scroll_container")
scroll_down(scroll_container)

In [138]:
# 높이가 어떻게 다른지 확인
scroll_container = driver.find_element(By.ID, "_pcmap_list_scroll_container")
print(driver.execute_script("return arguments[0].scrollTop", scroll_container))
print(driver.execute_script("return arguments[0].scrollHeight", scroll_container))

0
4198
600


In [5]:
foods = scroll_container.find_elements(By.CSS_SELECTOR, "ul > li")

In [17]:
# 기본 콘텐츠로 돌아오기
driver.switch_to.default_content()
driver.find_element(By.ID, "entryIframe").get_attribute("src")

'https://pcmap.place.naver.com/place/1912373073?entry=bmp&from=map&fromPanelNum=2&timestamp=202406131126'

---

### 유튜브 크롤링 검토

In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [19]:
# 크롬 창 열기
options = Options()
driver = webdriver.Chrome(options = options)

# 창모드 전체화면으로 크기 늘리기
driver.maximize_window()

# 로딩시간
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

# 유튜브 주소 접속하기
url = "https://www.youtube.com/watch?v=0BWScn_OWPk"
driver.get(url)

# 유튜브 로딩시간
time.sleep(5)

In [20]:
# n_scroll variable 만들기
n_scroll = 5

In [21]:
def scroll_down(n_scroll):
    """함수

    Args:
        n_scroll (integer): 스크롤 하는 횟수
    """
    # 첫번째 방법
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    for i in range(n_scroll):
        # 끝까지 스크롤 내리기
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(2)
        # 스크롤 내린 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    # 해결책2 - 페이지의 스크롤을 관련영상 5번째까지 올리기 -> 첫 스크롤까지만 데이터가 받아짐
    related_video = driver.find_elements(By.CSS_SELECTOR, '#dismissible')[5]
    driver.execute_script("arguments[0].scrollIntoView(true);", related_video)
    
    # 해결책3 - 스크롤 한번 더 내린다. 이번에 스크롤 길이만큼 내리며 
    for _ in range(10):
        time.sleep(5)
        # JavaScript를 사용하여 페이지 스크롤
        driver.execute_script("window.scrollBy(0, window.innerHeight);")
    
    # 댓글이 로드될 때까지 기다리기
    comments_section = WebDriverWait(driver, 10)
    comments_section.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#contents')))
scroll_down(n_scroll)

In [49]:
pause_script = """
var video = document.querySelector('video');
if (video) {
    video.pause();
}
"""
driver.execute_script(pause_script)

In [51]:
# 조금만 내리기
driver.execute_script("window.scrollTo(0, 200);")

In [52]:
n_scroll = 5

# 첫번째 방법
last_height = driver.execute_script("return document.documentElement.scrollHeight")
for i in range(n_scroll):
    # 끝까지 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(2)
    # 스크롤 내린 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [38]:
html = driver.find_element(By.TAG_NAME, "html")
driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", html)